# Enter State Farm

In [14]:
%matplotlib inline

import utils;
from utils import *
from IPython.display import FileLink

In [2]:
batch_size = 8

In [ ]:
%pwd

In [3]:
path = "../Input/sample/" # "../Input/"

## Create batches

In [4]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 593 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [5]:
trn_classes, val_classes, trn_labels, val_labels, filenames, val_filenames, test_filenames = get_classes(path)

Found 593 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Found 50 images belonging to 1 classes.


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.

In [6]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3, 256, 256)),
    Flatten(),
    Dense(10, activation='softmax')
])

As you can see below, this training is going nowhere...

In [7]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/2
53s - loss: 14.3344 - acc: 0.1017 - val_loss: 14.2136 - val_acc: 0.1100
Epoch 2/2
18s - loss: 14.5060 - acc: 0.1000 - val_loss: 14.5877 - val_acc: 0.0900


Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 256, 256)       12        
_________________________________________________________________
flatten_1 (Flatten)          (None, 196608)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1966090   
Total params: 1,966,102
Trainable params: 1,966,096
Non-trainable params: 6
_________________________________________________________________


Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [10]:
10*3*256*256

1966080

**Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. ** So let's check:

In [13]:
np.round(model.predict_generator(batches, steps=ceil(batches.n/batches.batch_size))[:10], 2)

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]], dtype=float32)

**Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate: **

In [14]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3, 256, 256)),
    Flatten(),
    Dense(10, activation='softmax')
])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/2
24s - loss: 2.5710 - acc: 0.2067 - val_loss: 2.2526 - val_acc: 0.2100
Epoch 2/2
35s - loss: 1.6234 - acc: 0.4833 - val_loss: 1.3879 - val_acc: 0.5300


In [15]:
model.optimizer.lr = 0.001

In [16]:
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=4, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/4
16s - loss: 1.1302 - acc: 0.6483 - val_loss: 1.3055 - val_acc: 0.5500
Epoch 2/4
43s - loss: 0.9504 - acc: 0.7268 - val_loss: 1.2055 - val_acc: 0.5700
Epoch 3/4
19s - loss: 0.6613 - acc: 0.8351 - val_loss: 0.9321 - val_acc: 0.7200
Epoch 4/4
17s - loss: 0.5645 - acc: 0.8783 - val_loss: 0.8792 - val_acc: 0.7300


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. **Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:**

In [19]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 100 images belonging to 10 classes.


In [21]:
val_res = [model.evaluate_generator(rnd_batches, steps=ceil(rnd_batches.n/rnd_batches.batch_size)) for i in range(10)]
np.round(val_res, 2)

array([[ 0.88,  0.73],
       [ 0.83,  0.76],
       [ 0.87,  0.74],
       [ 0.89,  0.7 ],
       [ 0.85,  0.75],
       [ 0.82,  0.75],
       [ 0.92,  0.72],
       [ 0.88,  0.73],
       [ 0.84,  0.76],
       [ 0.92,  0.71]])

**Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.**

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [8]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3, 256, 256)),
    Flatten(),
    Dense(10, activation='softmax', kernel_regularizer=l2(0.01))
])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/2
36s - loss: 2.8948 - acc: 0.1634 - val_loss: 3.3507 - val_acc: 0.2000
Epoch 2/2
40s - loss: 1.8911 - acc: 0.4067 - val_loss: 1.7976 - val_acc: 0.4400


In [9]:
model.optimizer.lr=0.001

In [10]:
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=4, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/4
36s - loss: 1.3875 - acc: 0.6084 - val_loss: 1.8020 - val_acc: 0.5100
Epoch 2/4
26s - loss: 1.1256 - acc: 0.7350 - val_loss: 1.4655 - val_acc: 0.6200
Epoch 3/4
21s - loss: 0.8757 - acc: 0.8450 - val_loss: 1.3776 - val_acc: 0.6200
Epoch 4/4
22s - loss: 0.7161 - acc: 0.8933 - val_loss: 1.1221 - val_acc: 0.7000


In [17]:
layers = model.layers
dense_idx = [idx for idx, layer in enumerate(layers) if type(layer)==Dense][0]
layers[dense_idx].kernel_regularizer = l2(0.1)

In [18]:
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/2
43s - loss: 0.6417 - acc: 0.9183 - val_loss: 1.2087 - val_acc: 0.7000
Epoch 2/2
27s - loss: 0.5263 - acc: 0.9567 - val_loss: 1.0962 - val_acc: 0.7300


Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [20]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3, 256, 256)),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax', kernel_regularizer=l2(0.01))
])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/2
76s - loss: 2.3246 - acc: 0.2967 - val_loss: 5.7354 - val_acc: 0.1100
Epoch 2/2
80s - loss: 1.6303 - acc: 0.5384 - val_loss: 2.0633 - val_acc: 0.3600


In [21]:
model.optimizer.lr=0.01
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=5, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Epoch 1/5
58s - loss: 1.1671 - acc: 0.6984 - val_loss: 1.4368 - val_acc: 0.5500
Epoch 2/5
84s - loss: 0.9416 - acc: 0.8051 - val_loss: 1.4365 - val_acc: 0.5500
Epoch 3/5
88s - loss: 0.7385 - acc: 0.8768 - val_loss: 1.1436 - val_acc: 0.7100
Epoch 4/5
81s - loss: 0.5632 - acc: 0.9401 - val_loss: 0.9215 - val_acc: 0.8300
Epoch 5/5
82s - loss: 0.5080 - acc: 0.9518 - val_loss: 0.8652 - val_acc: 0.7900


Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [10]:
def conv1(batches):
    model = Sequential([
        BatchNormalization(axis=1, input_shape=(3, 256, 256)),
        Conv2D(32, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3), strides=(3,3)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D((3,3), strides=(3,3)),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    
    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=2, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))
    model.optimizer.lr=0.001
    model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=5, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))
    
    return model

In [11]:
conv1(batches)

Epoch 1/2
686s - loss: 2.2094 - acc: 0.2882 - val_loss: 2.3336 - val_acc: 0.1300
Epoch 2/2
734s - loss: 1.0141 - acc: 0.7318 - val_loss: 2.1995 - val_acc: 0.2600
Epoch 1/5
838s - loss: 0.5581 - acc: 0.8718 - val_loss: 1.9454 - val_acc: 0.3400
Epoch 2/5
762s - loss: 0.2790 - acc: 0.9618 - val_loss: 1.4724 - val_acc: 0.5300
Epoch 3/5
3628s - loss: 0.1628 - acc: 0.9818 - val_loss: 1.0029 - val_acc: 0.7100
Epoch 4/5
767s - loss: 0.1361 - acc: 0.9852 - val_loss: 0.6793 - val_acc: 0.9000
Epoch 5/5
905s - loss: 0.1197 - acc: 0.9852 - val_loss: 0.5343 - val_acc: 0.9100


The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

#### Width shift: move the image left and right -

In [12]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

Found 1401 images belonging to 10 classes.
Epoch 1/2
1944s - loss: 2.1249 - acc: 0.2919 - val_loss: 2.4428 - val_acc: 0.1100
Epoch 2/2
1639s - loss: 1.4945 - acc: 0.5242 - val_loss: 1.7059 - val_acc: 0.4200
Epoch 1/5
1615s - loss: 1.1828 - acc: 0.6492 - val_loss: 1.0483 - val_acc: 0.7900
Epoch 2/5
2194s - loss: 0.9654 - acc: 0.7266 - val_loss: 0.8579 - val_acc: 0.7800
Epoch 3/5
4085s - loss: 0.8456 - acc: 0.7635 - val_loss: 0.8808 - val_acc: 0.7900
Epoch 4/5
1593s - loss: 0.7182 - acc: 0.8090 - val_loss: 0.5704 - val_acc: 0.8700
Epoch 5/5
1145s - loss: 0.6184 - acc: 0.8516 - val_loss: 0.6392 - val_acc: 0.7900


#### Height shift: move the image up and down -

In [13]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

Found 1401 images belonging to 10 classes.
Epoch 1/2
1176s - loss: 1.8384 - acc: 0.4183 - val_loss: 2.3572 - val_acc: 0.2600
Epoch 2/2
1156s - loss: 0.9802 - acc: 0.7110 - val_loss: 1.1929 - val_acc: 0.6000
Epoch 1/5
1167s - loss: 0.6405 - acc: 0.8239 - val_loss: 0.5394 - val_acc: 0.8500
Epoch 2/5
1187s - loss: 0.4766 - acc: 0.8857 - val_loss: 0.3585 - val_acc: 0.9400
Epoch 3/5
1895s - loss: 0.3868 - acc: 0.9162 - val_loss: 0.4166 - val_acc: 0.9000
Epoch 4/5
1802s - loss: 0.2990 - acc: 0.9432 - val_loss: 0.4091 - val_acc: 0.8800
Epoch 5/5
1966s - loss: 0.2761 - acc: 0.9460 - val_loss: 0.2518 - val_acc: 0.9600


** Random shear angles (max in radians) - **

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

**Rotation: max in degrees -**

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

** Channel shift: randomly changing the R,G,B colors - **

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

#### And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)
conv1(batches)

** At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.**

In [ ]:
model.optimizer.lr=0.0001
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=5, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Lucky we tried that - we starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, steps_per_epoch=ceil(batches.n/batches.batch_size), epochs=25, verbose=2,
                   validation_data=val_batches, validation_steps=ceil(val_batches.n/val_batches.batch_size))

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.